In [11]:
import pandas as pd
from os import getcwd 
from os.path import exists
getcwd()

'D:\\project\\MIT_glyco\\OGP-master'

In [2]:
flex_path = "./dynamine_results" # directory of flexibility results
angle_path = "./spider3_results" # directory of angle related information

In [15]:
# load O-GlcNAcylated site data
site_data = pd.read_csv("all_sites_with_colnames.csv") 

temp_name = 'temp'
for pro_name, pos_site in site_data[["name", "site"]].values: # protein name & positive site
    if pro_name != temp_name: # check whether the protein is same with before one
        if temp_name != 'temp':
            if not exists(f"./protein_sequence/{temp_name}.csv"):
                protein_df.to_csv(f"./protein_sequence/{temp_name}.csv", index_label=[pro_name])
            
        ## load flex data
        data = pd.read_fwf(f"{flex_path}/{pro_name}_backbone.pred", header=None)
        notes = data[:11] # this includes protein name and citation address
        # display(notes)
        ## split values from raw flexibility data
        flex = data[11:].copy().reset_index(drop=True)
        flex.columns = ["values"]
        flex['SEQ'] = flex['values'].apply(lambda x: x.split()[0])
        flex['flexibility'] = flex['values'].apply(lambda x: x.split()[1])

        ## load angle data
        angles = pd.read_csv(f"{angle_path}/{pro_name}.spd33")
        ## split values from raw angle data
        columns = angles.columns[0].split()
        angles.columns = ["values"]

        for i, column in enumerate(columns):
            angles[column] = angles['values'].apply(lambda x: x.split()[i])

        flex_values = flex.iloc[:,1:]
        angle_values = angles.iloc[:,3:]
        protein_df = pd.concat([flex_values, angle_values], axis=1)

        protein_df.index.name = pro_name
        
        protein_df["positive"] = 0
        protein_df.loc[pos_site-1, "positive"] = 1

        temp_name = pro_name
        
    else:
        protein_df.loc[pos_site-1, "positive"] = 1
protein_df.to_csv(f"./protein_sequence/{temp_name}.csv", index_label=[pro_name])